In [ ]:
import pandas as pd
import seaborn as sns
sns.set_style("whitegrid")
import pandas as pd
import numpy as np 
import scipy.stats as stats
from collections import Counter
import matplotlib.pyplot as plt
import umap
import matplotlib
import mygene
%matplotlib inline
import pickle
import sklearn
import random
import scanpy as sc
import torch
import torch.nn as nn
import torch.functional as F
# import sentence_transformers
plt.style.use('ggplot')
#plt.style.use('seaborn-v0_8-dark-palette')
plt.rcParams['axes.facecolor'] = 'white'
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
np.random.seed(202310)
# use hnswlib for NN classification
try:
    import hnswlib
    hnswlib_imported = True
except ImportError:
    hnswlib_imported = False
    print("hnswlib not installed! We highly recommend installing it for fast similarity search.")
    print("To install it, run: pip install hnswlib")
from scipy.stats import mode

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
sampled_adata = sc.read("/gpfs/gibbs/pi/zhao/tl688/board_heartcell/SCP1303/anndata/Cardiomyocyte_data_subsample0.1.h5ad")

In [ ]:
sc.pp.normalize_per_cell(sampled_adata)
sc.pp.log1p(sampled_adata)

sampled_adata.uns['log1p']['base'] = None
sc.pp.highly_variable_genes(sampled_adata, n_top_genes=2000)
sampled_adata = sampled_adata[:,sampled_adata.var.highly_variable]

In [ ]:
adata_comb = sampled_adata

In [ ]:
with open("/gpfs/gibbs/pi/zhao/tl688/cpsc_finalproject/genept_data/GenePT/ensem_emb_gpt3.5all.pickle", "rb") as fp:
    GPT_3_5_gene_embeddings = pickle.load(fp)
gene_names= list(adata_comb.var.index)
count_missing = 0
EMBED_DIM = 1536 # embedding dim from GPT-3.5
lookup_embed = np.zeros(shape=(len(gene_names),EMBED_DIM))
for i, gene in enumerate(gene_names):
    if gene in GPT_3_5_gene_embeddings:
        lookup_embed[i,:] = GPT_3_5_gene_embeddings[gene].flatten()
    else:
        count_missing+=1
lookup_embed.shape

In [ ]:
train_obs,test_obs,train_label,test_label = train_test_split(sampled_adata.obs_names, 
                                                    sampled_adata.obs.disease,
                                                    test_size=0.20, random_state=2023)

adata_train = sampled_adata[train_obs]
adata_test = sampled_adata[test_obs]

In [ ]:

lookup_embed = torch.FloatTensor(lookup_embed).to(device)

In [ ]:
label_encoder = LabelEncoder().fit(adata_train.obs.disease)

adata_train_train = adata_train

train_label = label_encoder.transform(adata_train_train.obs.disease)


X_train = torch.FloatTensor(adata_train_train.X.toarray())

train_label = torch.FloatTensor(train_label)

dataset = torch.utils.data.TensorDataset(X_train, train_label)

batch_size = 512

trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(lookup_embed.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, len(label_encoder.classes_))
        self.act = nn.ReLU()

    def forward(self, x, inputs):
        x = self.act(self.fc1(x))
        x = self.fc2(x)
        emb = torch.matmul(inputs, x)
        label_out = self.fc3(emb)
        return label_out,emb



net = Net().to(device)
dataset = torch.utils.data.TensorDataset(X_train.to(device), train_label.to(device))
trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                          shuffle=True)
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

def model_evaluation(model, data, labels):
    model.eval()
    data = data.to(device)
    labels = labels.to(device)
    outputs,_ = net(lookup_embed, data)

    _, predicted = torch.max(outputs, 1)
    
    return (predicted == labels).sum().item() / len(labels)

def model_output(model, data):
    model.eval()
    data = data.to(device)
    outputs,emb = net(lookup_embed, data)

    _, predicted = torch.max(outputs, 1)
    
    return predicted.cpu().numpy(), emb.cpu().detach().numpy()

In [ ]:
from pytorch_metric_learning import miners, losses
miner = miners.MultiSimilarityMiner()
loss_func = losses.TripletMarginLoss()

prev = 0
# here the validing section is not very important, since we pay more attention to generating distinguishable embeddings.
for epoch in range(40):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        labels = labels.long()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs,emb = net(lookup_embed, inputs)
        
        loss = criterion(outputs, labels) + 100 * loss_func(emb, labels)
#         loss = loss_func(emb, labels)
        loss.backward()
        optimizer.step()
        

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
print('Finished Training')

In [ ]:
_,genePT_w_emebed_train =  model_output(net, torch.FloatTensor(adata_train.X.toarray()))
_,genePT_w_emebed_test = model_output(net, torch.FloatTensor(adata_test.X.toarray()))

In [ ]:
adata_test.obsm['X_pca'] = genePT_w_emebed_test

In [ ]:
adata_test.obsm['X_genept'] = genePT_w_emebed_test

meanv = np.mean(adata_test[adata_test.obs['disease'] == 'NF'].obsm['X_genept'],axis=0)

meanv_ascend = np.mean(adata_test[adata_test.obs['disease'] == 'DCM'].obsm['X_genept'],axis=0)

import scipy

raw_cs = 1 - scipy.spatial.distance.cosine(meanv, meanv_ascend)

In [ ]:
np.random.seed(202310)
sc.tl.rank_genes_groups(sampled_adata, groupby='disease')

In [ ]:
disease = 'DCM'
control = 'NF'

In [ ]:
for i in sampled_adata.uns['rank_genes_groups']['names'][disease][0:10]:
    adata_test_new = adata_test.copy()
    adata_test_new[:,i].X = 0
    
    _,genePT_w_emebed_test = model_output(net, torch.FloatTensor(adata_test_new.X.toarray()))
    adata_test_new.obsm['X_genept'] = genePT_w_emebed_test
    meanv = np.mean(adata_test_new[adata_test_new.obs['disease'] == control].obsm['X_genept'],axis=0)
    meanv_ascend = np.mean(adata_test_new[adata_test_new.obs['disease'] == disease].obsm['X_genept'],axis=0)
#     print(i)
    print(1 - scipy.spatial.distance.cosine(meanv, meanv_ascend) - raw_cs)
    